In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import copy
import random
import cv2
import torch
import numpy as np
from torch import nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from ipywidgets import interact

random_seed = 2024

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
def list_image_files(data_dir, sub_dir):
    image_format = ["jpg"]

    image_files = []
    images_dir = os.path.join(data_dir, sub_dir)
    for file_path in os.listdir(images_dir):
        if file_path.split(".")[-1] in image_format:
            image_files.append(os.path.join(sub_dir, file_path))
    return image_files

In [ ]:
data_dir = "/content/drive/MyDrive/tomatos/train/"

Step1_list = list_image_files(data_dir, "Step1")
Step2_list = list_image_files(data_dir, "Step2")
Step3_list = list_image_files(data_dir, "Step3")
Step4_list = list_image_files(data_dir, "Step4")
Step5_list = list_image_files(data_dir, "Step5")

In [ ]:
def get_RGB_image(data_dir, file_name):
    image_file = os.path.join(data_dir, file_name)
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
min_num_files = min(len(Step1_list), len(Step2_list), len(Step3_list), len(Step4_list), len(Step5_list))

@interact(index=(0, min_num_files-1))
def show_samples(index=0):
    S1_image = get_RGB_image(data_dir, Step1_list[index])
    S2_image = get_RGB_image(data_dir, Step2_list[index])
    S3_image = get_RGB_image(data_dir, Step3_list[index])
    S4_image = get_RGB_image(data_dir, Step4_list[index])
    S5_image = get_RGB_image(data_dir, Step5_list[index])


    plt.figure(figsize=(16, 12))
    plt.subplot(131)
    plt.title("S1")
    plt.imshow(S1_image)
    plt.subplot(132)
    plt.title("S2")
    plt.imshow(S2_image)
    plt.subplot(133)
    plt.title("S3")
    plt.imshow(S3_image)
    plt.subplot(134)
    plt.title("S4")
    plt.imshow(S4_image)
    plt.subplot(135)
    plt.title("S5")
    plt.imshow(S5_image)
    plt.tight_layout()

In [ ]:
print(len(Step1_list))
print(len(Step2_list))
print(len(Step3_list))
print(len(Step4_list))
print(len(Step5_list))

In [ ]:
train_data_dir = "/content/drive/MyDrive/tomatos/train/"
class_list = ["Step1", "Step2", "Step3", "Step4", "Step5"]

In [ ]:
from PIL import Image

class ChestDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        Step1 = list_image_files(data_dir, "Step1")
        Step2 = list_image_files(data_dir, "Step2")
        Step3 = list_image_files(data_dir, "Step3")
        Step4 = list_image_files(data_dir, "Step4")
        Step5 = list_image_files(data_dir, "Step5")

        self.files_path = Step1 + Step2 + Step3 + Step4 + Step5
        self.transform = transform

    def __len__(self):
        return len(self.files_path)

    def __getitem__(self, index):
        image_file = os.path.join(self.data_dir, self.files_path[index])
        image = Image.open(image_file).convert("RGB")  # PIL로 이미지 열기 및 BGR에서 RGB로 변환

        target = class_list.index(self.files_path[index].split(os.sep)[0])  # 타겟을 클래스 인덱스로 설정

        if self.transform:
            image = self.transform(image)

        return image, target




In [ ]:
dset = Chest_dataset(train_data_dir)

In [ ]:
index = 1
plt.title(class_list[dset[index]["target"]])
plt.imshow(dset[index]["image"])

In [ ]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

In [ ]:
train_dset = Chest_dataset(train_data_dir, transformer)

In [ ]:
index = 200
image = train_dset[index]["image"]
label = train_dset[index]["target"]

In [ ]:
print(image.shape, label)

In [ ]:
def build_dataloader(train_data_dir, val_data_dir):
    dataloaders = {}
    train_dset = Chest_dataset(train_data_dir, transformer)
    dataloaders["train"] = DataLoader(train_dset, batch_size=4, shuffle=True, drop_last=True)

    val_dset = Chest_dataset(val_data_dir, transformer)
    dataloaders["val"] = DataLoader(val_dset, batch_size=1, shuffle=False, drop_last=False)
    return dataloaders

In [ ]:
train_data_dir = "/content/drive/MyDrive/tomatos/train/"
val_data_dir = "/content/drive/MyDrive/tomatos/test/"
dataloaders = build_dataloader(train_data_dir, val_data_dir)

In [ ]:
model = models.vgg19(pretrained=True)

In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224), batch_size=1, device="cpu")

In [ ]:
def build_vgg19_based_model(device_name='cpu'):
    device = torch.device(device_name)
    model = models.vgg19(pretrained=True)
    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, len(class_list)),
        nn.Softmax(dim=1)
    )
    return model.to(device)

In [ ]:
model = build_vgg19_based_model(device_name='cpu')

In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224), batch_size=1, device="cpu")

In [ ]:
loss_func = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr= 1E-3, momentum=0.9)

In [ ]:
@torch.no_grad()
def get_accuracy(image, target, model):
    batch_size = image.shape[0]
    prediction = model(image)
    _, pred_label = torch.max(prediction, dim=1)
    is_correct = (pred_label == target)
    return is_correct.cpu().numpy().sum() / batch_size

In [ ]:
device = torch.device("cpu")

In [ ]:
ddevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data_dir = "/content/drive/MyDrive/tomatos/train/"
val_data_dir = "/content/drive/MyDrive/tomatos/test/"

dataloaders = build_dataloader(train_data_dir, val_data_dir)
model = build_vgg19_based_model()
loss_func = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr= 1E-3, momentum=0.9)


In [ ]:
data_data = "../DATASET/Classification/test/"
class_list = ["Normal", "Covid", "Viral Pneumonia"]

test_normals_list = list_image_files(data_dir, "Normal")
test_covids_list = list_image_files(data_dir, "Covid")
test_pneumonias_list = list_image_files(data_dir, "Viral Pneumonia")

In [ ]:
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms

# 클래스 리스트 업데이트
class_list = ['Step1', 'Step2', 'Step3', 'Step4', 'Step5']

# 데이터셋 클래스 정의
class ChestDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        Step1 = list_image_files(data_dir, "Step1")
        Step2 = list_image_files(data_dir, "Step2")
        Step3 = list_image_files(data_dir, "Step3")
        Step4 = list_image_files(data_dir, "Step4")
        Step5 = list_image_files(data_dir, "Step5")

        self.files_path = Step1 + Step2 + Step3 + Step4 + Step5
        self.transform = transform

    def __len__(self):
        return len(self.files_path)

    def __getitem__(self, index):
        image_file = os.path.join(self.data_dir, self.files_path[index])
        image = Image.open(image_file).convert("RGB")  # PIL로 이미지 열기 및 BGR에서 RGB로 변환

        target = class_list.index(self.files_path[index].split(os.sep)[0])  # 타겟을 클래스 인덱스로 설정

        if self.transform:
            image = self.transform(image)

        return image, target

# 학습 함수 정의
def train_one_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_accuracy = correct / total
    return epoch_loss, epoch_accuracy

# 데이터 경로 리스트를 가져오는 함수 정의
def list_image_files(data_dir, sub_dir):
    image_format = [".jpg", ".jpeg", ".png"]
    image_files = []
    images_dir = os.path.join(data_dir, sub_dir)
    for file_path in os.listdir(images_dir):
        if file_path.split(".")[-1] in image_format:
            image_files.append(os.path.join(sub_dir, file_path))
    return image_files

# GPU 사용 가능 여부 확인 및 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리 및 데이터로더 생성
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ChestDataset(train_data_dir, transform=transform)
val_dataset = ChestDataset(val_data_dir, transform=transform)

train_loader = DataLoader(train_dataset_dir, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset_dir, batch_size=16)

# 모델 생성 및 출력 레이어 수정
num_classes = 5
model = models.vgg19(pretrained=True)
model.classifier[-1] = nn.Linear(4096, num_classes)  # 출력 레이어 수정
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 학습 진행
num_epochs = 10
best_accuracy = 0.0

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_one_epoch(model, train_loader, optimizer, criterion, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss}, Train Accuracy: {train_accuracy}")

    # 검증 성능 측정 및 모델 저장
    with torch.no_grad():
        val_loss, val_accuracy = train_one_epoch(model, val_loader, optimizer, criterion, device)
        print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss}, Val Accuracy: {val_accuracy}")
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_model.pth")

print(f"Best Validation Accuracy: {best_accuracy}")
